In [13]:
'''
Author: xushengyichn 54436848+xushengyichn@users.noreply.github.com
Date: 2023-01-24 14:28:48
LastEditors: xushengyichn 54436848+xushengyichn@users.noreply.github.com
LastEditTime: 2023-01-25 18:51:17
FilePath: \20230124优化问题\bayesian.ipynb
Description: 创建贝叶斯优化，尝试优化6个TMD的问题

Copyright (c) 2023 by xushengyichn 54436848+xushengyichn@users.noreply.github.com, All Rights Reserved. 
'''


'\nAuthor: xushengyichn 54436848+xushengyichn@users.noreply.github.com\nDate: 2023-01-24 14:28:48\nLastEditors: xushengyichn 54436848+xushengyichn@users.noreply.github.com\nLastEditTime: 2023-01-25 18:51:17\nFilePath: \x8230124优化问题\x08ayesian.ipynb\nDescription: 创建贝叶斯优化，尝试优化6个TMD的问题\n\nCopyright (c) 2023 by xushengyichn 54436848+xushengyichn@users.noreply.github.com, All Rights Reserved. \n'

In [14]:
#%% 1. 导入库
import time
import matlab
import matlab.engine
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.gaussian_process.kernels import Matern
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.model_selection import cross_val_score,KFold,cross_validate
from sklearn.linear_model import LogisticRegression

In [15]:
#%% 2. 定义函数
eng = matlab.engine.start_matlab()
def black_box_function(mTMD1,mTMD2,mTMD3,mTMD4,mTMD5,fTMD1,fTMD2,fTMD3,fTMD4,fTMD5,fTMD6,dTMD1,dTMD2,dTMD3,dTMD4,dTMD5,dTMD6,xTMD1,xTMD2,xTMD3,xTMD4,xTMD5,xTMD6):
    total_tmd_mass_ratio = 0.02 # 总质量比 The total mass ratio
    mass_six_span = 10007779.7 # 深中通道非通航桥六跨连续梁质量 The mass of 6-span continuous beam of the non-navigational bridge of the Zhenzhong-Link
    total_tmd_mass = total_tmd_mass_ratio * mass_six_span # 总质量 The total mass
    single_tmd_mass=total_tmd_mass/6 # 单个TMD质量 The mass of a single TMD
    mTMD1=0.5*single_tmd_mass+mTMD1*0.5*single_tmd_mass # 质量 The mass mTMD1
    mTMD2=0.5*single_tmd_mass+mTMD2*0.5*single_tmd_mass # 质量 The mass mTMD2
    mTMD3=0.5*single_tmd_mass+mTMD3*0.5*single_tmd_mass # 质量 The mass mTMD3
    mTMD4=0.5*single_tmd_mass+mTMD4*0.5*single_tmd_mass # 质量 The mass mTMD4
    mTMD5=0.5*single_tmd_mass+mTMD5*0.5*single_tmd_mass # 质量 The mass mTMD5
    mTMD6= total_tmd_mass - mTMD1 - mTMD2 - mTMD3 - mTMD4 - mTMD5 # 质量 The mass mTMD6
    
    fTMD1=0.7+fTMD1*1.3 # 频率 The frequency fTMD1
    fTMD2=0.7+fTMD2*1.3 # 频率 The frequency fTMD2
    fTMD3=0.7+fTMD3*1.3 # 频率 The frequency fTMD3
    fTMD4=0.7+fTMD4*1.3 # 频率 The frequency fTMD4
    fTMD5=0.7+fTMD5*1.3 # 频率 The frequency fTMD5
    fTMD6=0.7+fTMD6*1.3 # 频率 The frequency fTMD6
    
    dTMD1=0.05+dTMD1*0.15 # 阻尼比 The damping ratio dTMD1
    dTMD2=0.05+dTMD2*0.15 # 阻尼比 The damping ratio dTMD2
    dTMD3=0.05+dTMD3*0.15 # 阻尼比 The damping ratio dTMD3
    dTMD4=0.05+dTMD4*0.15 # 阻尼比 The damping ratio dTMD4
    dTMD5=0.05+dTMD5*0.15 # 阻尼比 The damping ratio dTMD5
    dTMD6=0.05+dTMD6*0.15 # 阻尼比 The damping ratio dTMD6
    
    
    xTMD1=xTMD1*660 # TMD1的x坐标 The x-coordinate of TMD1
    xTMD2=xTMD2*660 # TMD2的x坐标 The x-coordinate of TMD2
    xTMD3=xTMD3*660 # TMD3的x坐标 The x-coordinate of TMD3
    xTMD4=xTMD4*660 # TMD4的x坐标 The x-coordinate of TMD4
    xTMD5=xTMD5*660 # TMD5的x坐标 The x-coordinate of TMD5
    xTMD6=xTMD6*660 # TMD6的x坐标 The x-coordinate of TMD6
    
    
    t_length=matlab.double(100) # 时间长度 The time length
    number_of_modes_to_control=matlab.double([1,2,3,4,5,6]) # 控制模态 The controlled modes
    number_of_modes_to_consider=10 # 考虑模态 The considered modes
    number_of_tmds=6 # TMD数量 The number of TMDs
    modal_damping_ratios=np.ones((1,number_of_modes_to_consider))*0.003 # 模态阻尼比 The modal damping ratios
    
    result = -eng.b_0_6_tmd(number_of_modes_to_control,number_of_modes_to_consider,number_of_tmds,modal_damping_ratios,t_length,mTMD1,mTMD2,mTMD3,mTMD4,mTMD5,fTMD1,fTMD2,fTMD3,fTMD4,fTMD5,fTMD6,dTMD1,dTMD2,dTMD3,dTMD4,dTMD5,dTMD6,xTMD1,xTMD2,xTMD3,xTMD4,xTMD5,xTMD6,total_tmd_mass)
    return result

In [16]:
#%% 3. 定义优化器
optimizer = BayesianOptimization(
    f=black_box_function,
    pbounds={"mTMD1": (0,1), 
             "mTMD2": (0,1),
             "mTMD3": (0,1),
             "mTMD4": (0,1),
             "mTMD5": (0,1),
             "fTMD1": (0,1),
             "fTMD2": (0,1),
             "fTMD3": (0,1),
             "fTMD4": (0,1),
             "fTMD5": (0,1),
             "fTMD6": (0,1),
             "dTMD1": (0,1),
             "dTMD2": (0,1),
             "dTMD3": (0,1),
             "dTMD4": (0,1),
             "dTMD5": (0,1),
             "dTMD6": (0,1),
             "xTMD1": (0,1),
             "xTMD2": (0,1),
             "xTMD3": (0,1),
             "xTMD4": (0,1),
             "xTMD5": (0,1),
             "xTMD6": (0,1),
             },
    verbose=2,  # verbose = 1 prints only when a maximum is observed, verbose = 0 is silent
    random_state=1,
)


optimizer.probe(
    params={ "mTMD1": 1, 
             "mTMD2": 1,
             "mTMD3": 1,
             "mTMD4": 1,
             "mTMD5": 1,
             "fTMD1": 0.1,
             "fTMD2": 0.153846,
             "fTMD3": 0.276923,
             "fTMD4": 0.446153,
             "fTMD5": 0.623077,
             "fTMD6": 0.769231,
             "dTMD1": 0.2,
             "dTMD2": 0.2,
             "dTMD3": 0.2,
             "dTMD4": 0.2,
             "dTMD5": 0.2,
             "dTMD6": 0.2,
             "xTMD1": 0.4181818,
             "xTMD2": 0.9181818,
             "xTMD3": 0.9212121,
             "xTMD4": 0.5969697,
             "xTMD5": 0.2515152,
             "xTMD6": 0.5848485,},
    lazy=True,
)



In [17]:
optimizer.maximize(init_points=0, n_iter=0)


|   iter    |  target   |   dTMD1   |   dTMD2   |   dTMD3   |   dTMD4   |   dTMD5   |   dTMD6   |   fTMD1   |   fTMD2   |   fTMD3   |   fTMD4   |   fTMD5   |   fTMD6   |   mTMD1   |   mTMD2   |   mTMD3   |   mTMD4   |   mTMD5   |   xTMD1   |   xTMD2   |   xTMD3   |   xTMD4   |   xTMD5   |   xTMD6   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | -0.2433   | 0.2       | 0.2       | 0.2       | 0.2       | 0.2       | 0.2       | 0.1       | 0.1538    | 0.2769    | 0.4462    | 0.6231    | 0.7692    | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 0.4182    | 0.9182    | 0.9212    | 0.597     | 0.2515    | 0.5848    |


In [18]:
optimizer.maximize(
        init_points=0,
        n_iter=100,
        acq="ei",  # Expected Improvement.
        # kappa=7.2088954429132,
        # xi=0.0012265019946638392,
        xi=0.1,
        # What follows are GP regressor parameters
        kernel=Matern(nu=1.5),
        # alpha=1,
        # alpha=2.2492215663634233e-10,
        alpha=8.960456511761172e-06,
        normalize_y=True,
        n_restarts_optimizer=5,
)

|   iter    |  target   |   dTMD1   |   dTMD2   |   dTMD3   |   dTMD4   |   dTMD5   |   dTMD6   |   fTMD1   |   fTMD2   |   fTMD3   |   fTMD4   |   fTMD5   |   fTMD6   |   mTMD1   |   mTMD2   |   mTMD3   |   mTMD4   |   mTMD5   |   xTMD1   |   xTMD2   |   xTMD3   |   xTMD4   |   xTMD5   |   xTMD6   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 2         | -1.302    | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 1.0       | 0.0       | 0.0       | 0.0       | 0.0       | 0.0       | 0.0       | 0.0       | 0.0       | 0.0       | 0.0       | 0.0       | 1.0       | 1.0       | 1.0       |
| 3         | -1.047    | 0.01124   | 0.08676   | 0.0551    | 0.1873    | 0.3293    | 0.5466  

In [19]:
optimizer.max['target']
print(optimizer.max)

{'target': -0.2432825494279443, 'params': {'dTMD1': 0.2, 'dTMD2': 0.2, 'dTMD3': 0.2, 'dTMD4': 0.2, 'dTMD5': 0.2, 'dTMD6': 0.2, 'fTMD1': 0.1, 'fTMD2': 0.153846, 'fTMD3': 0.276923, 'fTMD4': 0.446153, 'fTMD5': 0.623077, 'fTMD6': 0.769231, 'mTMD1': 1.0, 'mTMD2': 1.0, 'mTMD3': 1.0, 'mTMD4': 1.0, 'mTMD5': 1.0, 'xTMD1': 0.4181818, 'xTMD2': 0.9181818, 'xTMD3': 0.9212121, 'xTMD4': 0.5969697, 'xTMD5': 0.2515152, 'xTMD6': 0.5848485}}


In [20]:
optimizer.probe(
    params={ "mTMD1": 1, 
             "mTMD2": 1,
             "mTMD3": 1,
             "mTMD4": 1,
             "mTMD5": 1,
             "fTMD1": 0.1,
             "fTMD2": 0.153846,
             "fTMD3": 0.276923,
             "fTMD4": 0.446153,
             "fTMD5": 0.623077,
             "fTMD6": 0.769231,
             "dTMD1": 0.2,
             "dTMD2": 0.2,
             "dTMD3": 0.2,
             "dTMD4": 0.2,
             "dTMD5": 0.2,
             "dTMD6": 0.2,
             "xTMD1": 0.4181818,
             "xTMD2": 0.9181818,
             "xTMD3": 0.9212121,
             "xTMD4": 0.5969697,
             "xTMD5": 0.2515152,
             "xTMD6": 0.5848485,},
    lazy=True,
)

In [21]:
optimizer.maximize(init_points=0, n_iter=0)

|   iter    |  target   |   dTMD1   |   dTMD2   |   dTMD3   |   dTMD4   |   dTMD5   |   dTMD6   |   fTMD1   |   fTMD2   |   fTMD3   |   fTMD4   |   fTMD5   |   fTMD6   |   mTMD1   |   mTMD2   |   mTMD3   |   mTMD4   |   mTMD5   |   xTMD1   |   xTMD2   |   xTMD3   |   xTMD4   |   xTMD5   |   xTMD6   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 102       | -1.26     | 0.2893    | 0.6279    | 0.9941    | 0.6318    | 0.8875    | 0.1214    | 0.1505    | 0.633     | 0.8941    | 0.8841    | 0.02891   | 0.6449    | 0.8796    | 0.1635    | 0.8043    | 0.4069    | 0.7729    | 0.9885    | 0.8906    | 0.1689    | 0.08772   | 0.1704    | 0.1409    |


In [22]:
optimizer.maximize(
        init_points=0,
        n_iter=100,
        acq="ei",  # Expected Improvement.
        # kappa=7.2088954429132,
        xi=0.0012265019946638392,
        # What follows are GP regressor parameters
        kernel=Matern(nu=1.5),
        # alpha=1,
        # alpha=2.2492215663634233e-10,
        alpha=8.960456511761172e-06,
        normalize_y=True,
        n_restarts_optimizer=5,
)

|   iter    |  target   |   dTMD1   |   dTMD2   |   dTMD3   |   dTMD4   |   dTMD5   |   dTMD6   |   fTMD1   |   fTMD2   |   fTMD3   |   fTMD4   |   fTMD5   |   fTMD6   |   mTMD1   |   mTMD2   |   mTMD3   |   mTMD4   |   mTMD5   |   xTMD1   |   xTMD2   |   xTMD3   |   xTMD4   |   xTMD5   |   xTMD6   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 103       | -0.8153   | 0.4173    | 0.7546    | 0.1972    | 0.04668   | 0.607     | 0.8661    | 0.3989    | 0.4976    | 0.6129    | 0.1879    | 0.5601    | 0.4225    | 0.1851    | 0.5696    | 0.6986    | 0.5334    | 0.2645    | 0.4256    | 0.9646    | 0.5246    | 0.9087    | 0.4948    | 0.003786  |
| 104       | -0.9343   | 0.8193    | 0.06335   | 0.742     | 0.3644    | 0.0001151 | 0.9527  

In [23]:
optimizer.max['target']
print(optimizer.max)

{'target': -0.2432825494279443, 'params': {'dTMD1': 0.2, 'dTMD2': 0.2, 'dTMD3': 0.2, 'dTMD4': 0.2, 'dTMD5': 0.2, 'dTMD6': 0.2, 'fTMD1': 0.1, 'fTMD2': 0.153846, 'fTMD3': 0.276923, 'fTMD4': 0.446153, 'fTMD5': 0.623077, 'fTMD6': 0.769231, 'mTMD1': 1.0, 'mTMD2': 1.0, 'mTMD3': 1.0, 'mTMD4': 1.0, 'mTMD5': 1.0, 'xTMD1': 0.4181818, 'xTMD2': 0.9181818, 'xTMD3': 0.9212121, 'xTMD4': 0.5969697, 'xTMD5': 0.2515152, 'xTMD6': 0.5848485}}


In [24]:
optimizer.maximize(
        init_points=0,
        n_iter=500,
        acq="ei",  # Expected Improvement.
        # kappa=7.2088954429132,
        xi=0.0012265019946638392,
        # What follows are GP regressor parameters
        kernel=Matern(nu=1.5),
        # alpha=1,
        # alpha=2.2492215663634233e-10,
        alpha=8.960456511761172e-06,
        normalize_y=True,
        n_restarts_optimizer=5,
)

|   iter    |  target   |   dTMD1   |   dTMD2   |   dTMD3   |   dTMD4   |   dTMD5   |   dTMD6   |   fTMD1   |   fTMD2   |   fTMD3   |   fTMD4   |   fTMD5   |   fTMD6   |   mTMD1   |   mTMD2   |   mTMD3   |   mTMD4   |   mTMD5   |   xTMD1   |   xTMD2   |   xTMD3   |   xTMD4   |   xTMD5   |   xTMD6   |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
| 203       | -0.7656   | 0.5004    | 0.4219    | 0.5136    | 0.3935    | 0.03806   | 0.1534    | 0.5499    | 0.03095   | 0.2174    | 0.5044    | 0.668     | 0.09879   | 0.9023    | 0.7677    | 0.6256    | 0.6585    | 0.8516    | 0.2969    | 0.595     | 0.1018    | 0.4548    | 0.8524    | 0.5193    |
| 204       | -0.8402   | 0.8617    | 0.4045    | 0.4983    | 0.11      | 0.08217   | 0.5079  